In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
class Arg:
    arch = 'resnet18'
    data = '/data/imagenet/ILSVRC/Data/CLS-LOC/'
    batch_size = 128
    workers = 4
    start_epoch = 0
    epochs = 90
    lr = 0.1
    momentum = 0.9
    weight_decay = 1e-4
    print_freq = 10
    resume = ''
    world_size = -1
    rank = -1
    dist_url = 'tcp://224.66.41.62:23456'
    seed = None
    dist_backend = 'nccl'
    gpu = None
    distributed = False
    pretrained = False
    evaluate = False
    multiprocessing_distributed = False
args = Arg()

In [3]:
best_acc1 = 0

In [4]:
ngpus_per_node = torch.cuda.device_count()

In [5]:
def train(train_loader, model, criterion, optimizer, epoch, args):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            images = images.cuda(args.gpu, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(args.gpu, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i)


def validate(val_loader, model, criterion, args):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if args.gpu is not None:
                images = images.cuda(args.gpu, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(args.gpu, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [6]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [7]:
def adjust_learning_rate(optimizer, epoch, args):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [10]:
def main_worker(gpu, ngpus_per_node, args):
    global best_acc1
    args.gpu = gpu

    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))

    if args.distributed:
        if args.dist_url == "env://" and args.rank == -1:
            args.rank = int(os.environ["RANK"])
        if args.multiprocessing_distributed:
            # For multiprocessing distributed training, rank needs to be the
            # global rank among all the processes
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                world_size=args.world_size, rank=args.rank)
    # create model
    if args.pretrained:
        print("=> using pre-trained model '{}'".format(args.arch))
        model = models.__dict__[args.arch](pretrained=True)
    else:
        print("=> creating model '{}'".format(args.arch))
        model = models.__dict__[args.arch]()

    if not torch.cuda.is_available():
        print('using CPU, this will be slow')
    elif args.distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            model.cuda(args.gpu)
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int((args.workers + ngpus_per_node - 1) / ngpus_per_node)
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        else:
            model.cuda()
            # DistributedDataParallel will divide and allocate batch_size to all
            # available GPUs if device_ids are not set
            model = torch.nn.parallel.DistributedDataParallel(model)
    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        model = model.cuda(args.gpu)
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        if args.arch.startswith('alexnet') or args.arch.startswith('vgg'):
            model.features = torch.nn.DataParallel(model.features)
            model.cuda()
        else:
            model = torch.nn.DataParallel(model).cuda()

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda(args.gpu)

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            if args.gpu is None:
                checkpoint = torch.load(args.resume)
            else:
                # Map model to be loaded to specified single gpu.
                loc = 'cuda:{}'.format(args.gpu)
                checkpoint = torch.load(args.resume, map_location=loc)
            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_acc1']
            if args.gpu is not None:
                # best_acc1 may be from a checkpoint from a different GPU
                best_acc1 = best_acc1.to(args.gpu)
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    traindir = '/home/jtang/Desktop/CAE_real'
    valdir = '~/Downloads/ILSVRC/Data/CLS-LOC/val'
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_dataset = datasets.ImageFolder(
        traindir,
        transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None),
        num_workers=args.workers, pin_memory=True, sampler=train_sampler)

    val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True)

    if args.evaluate:
        validate(val_loader, model, criterion, args)
        return

    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            train_sampler.set_epoch(epoch)
        adjust_learning_rate(optimizer, epoch, args)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, args)

        # evaluate on validation set
        acc1 = validate(val_loader, model, criterion, args)

        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        if not args.multiprocessing_distributed or (args.multiprocessing_distributed
                and args.rank % ngpus_per_node == 0):
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': args.arch,
                'state_dict': model.state_dict(),
                'best_acc1': best_acc1,
                'optimizer' : optimizer.state_dict(),
            }, is_best)

In [11]:
main_worker(args.gpu, ngpus_per_node, args)

=> creating model 'resnet18'
Epoch: [0][   0/1002]	Time  4.373 ( 4.373)	Data  1.738 ( 1.738)	Loss 7.0813e+00 (7.0813e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.78 (  0.78)
Epoch: [0][  10/1002]	Time  0.369 ( 0.715)	Data  0.000 ( 0.159)	Loss 7.2769e+00 (7.1516e+00)	Acc@1   0.00 (  0.14)	Acc@5   0.78 (  0.78)
Epoch: [0][  20/1002]	Time  0.380 ( 0.546)	Data  0.000 ( 0.095)	Loss 7.1274e+00 (7.2459e+00)	Acc@1   0.00 (  0.07)	Acc@5   0.00 (  0.56)
Epoch: [0][  30/1002]	Time  0.367 ( 0.491)	Data  0.000 ( 0.072)	Loss 7.1177e+00 (7.2060e+00)	Acc@1   0.00 (  0.08)	Acc@5   0.00 (  0.50)
Epoch: [0][  40/1002]	Time  0.331 ( 0.459)	Data  0.000 ( 0.061)	Loss 7.0281e+00 (7.1758e+00)	Acc@1   0.00 (  0.06)	Acc@5   0.00 (  0.50)
Epoch: [0][  50/1002]	Time  0.376 ( 0.438)	Data  0.000 ( 0.053)	Loss 7.0392e+00 (7.1373e+00)	Acc@1   0.00 (  0.05)	Acc@5   0.00 (  0.46)
Epoch: [0][  60/1002]	Time  0.345 ( 0.427)	Data  0.000 ( 0.049)	Loss 6.9519e+00 (7.1066e+00)	Acc@1   0.00 (  0.05)	Acc@5   0.78 (  0.51)
Epoch: [0][ 

Epoch: [0][ 600/1002]	Time  0.367 ( 0.370)	Data  0.000 ( 0.040)	Loss 6.6250e+00 (6.8502e+00)	Acc@1   0.00 (  0.27)	Acc@5   0.78 (  1.30)
Epoch: [0][ 610/1002]	Time  0.405 ( 0.370)	Data  0.000 ( 0.040)	Loss 6.6569e+00 (6.8483e+00)	Acc@1   1.56 (  0.27)	Acc@5   2.34 (  1.31)
Epoch: [0][ 620/1002]	Time  0.380 ( 0.370)	Data  0.000 ( 0.040)	Loss 6.7859e+00 (6.8465e+00)	Acc@1   1.56 (  0.28)	Acc@5   2.34 (  1.33)
Epoch: [0][ 630/1002]	Time  0.363 ( 0.370)	Data  0.000 ( 0.040)	Loss 6.6533e+00 (6.8443e+00)	Acc@1   1.56 (  0.28)	Acc@5   5.47 (  1.34)
Epoch: [0][ 640/1002]	Time  0.367 ( 0.370)	Data  0.000 ( 0.039)	Loss 6.6230e+00 (6.8419e+00)	Acc@1   0.00 (  0.28)	Acc@5   6.25 (  1.37)
Epoch: [0][ 650/1002]	Time  0.397 ( 0.370)	Data  0.000 ( 0.039)	Loss 6.7143e+00 (6.8402e+00)	Acc@1   0.00 (  0.28)	Acc@5   0.78 (  1.37)
Epoch: [0][ 660/1002]	Time  0.363 ( 0.370)	Data  0.000 ( 0.039)	Loss 6.6349e+00 (6.8378e+00)	Acc@1   1.56 (  0.29)	Acc@5   2.34 (  1.39)
Epoch: [0][ 670/1002]	Time  0.367 ( 0.370

Test: [240/391]	Time  0.899 ( 0.369)	Loss 6.4481e+00 (6.4904e+00)	Acc@1   0.78 (  0.86)	Acc@5   3.91 (  3.42)
Test: [250/391]	Time  0.159 ( 0.367)	Loss 6.5189e+00 (6.4959e+00)	Acc@1   0.78 (  0.83)	Acc@5   3.12 (  3.42)
Test: [260/391]	Time  1.215 ( 0.369)	Loss 6.8476e+00 (6.5010e+00)	Acc@1   0.00 (  0.81)	Acc@5   0.00 (  3.34)
Test: [270/391]	Time  0.111 ( 0.367)	Loss 6.9661e+00 (6.5094e+00)	Acc@1   0.00 (  0.78)	Acc@5   0.00 (  3.27)
Test: [280/391]	Time  1.086 ( 0.370)	Loss 6.4452e+00 (6.5152e+00)	Acc@1   0.00 (  0.77)	Acc@5   0.00 (  3.21)
Test: [290/391]	Time  0.113 ( 0.367)	Loss 6.8306e+00 (6.5206e+00)	Acc@1   0.00 (  0.75)	Acc@5   0.00 (  3.15)
Test: [300/391]	Time  0.988 ( 0.367)	Loss 6.6857e+00 (6.5267e+00)	Acc@1   0.00 (  0.77)	Acc@5   0.78 (  3.16)
Test: [310/391]	Time  0.126 ( 0.366)	Loss 6.6750e+00 (6.5332e+00)	Acc@1   0.78 (  0.75)	Acc@5   5.47 (  3.10)
Test: [320/391]	Time  1.054 ( 0.368)	Loss 6.6642e+00 (6.5391e+00)	Acc@1   0.00 (  0.73)	Acc@5   0.00 (  3.02)
Test: [330

Epoch: [1][ 470/1002]	Time  0.358 ( 0.368)	Data  0.000 ( 0.028)	Loss 6.3892e+00 (6.4999e+00)	Acc@1   0.00 (  0.85)	Acc@5   1.56 (  3.62)
Epoch: [1][ 480/1002]	Time  0.357 ( 0.369)	Data  0.000 ( 0.028)	Loss 6.3653e+00 (6.4974e+00)	Acc@1   1.56 (  0.85)	Acc@5   3.12 (  3.64)
Epoch: [1][ 490/1002]	Time  0.367 ( 0.369)	Data  0.000 ( 0.028)	Loss 6.4601e+00 (6.4962e+00)	Acc@1   0.00 (  0.86)	Acc@5   2.34 (  3.66)
Epoch: [1][ 500/1002]	Time  0.365 ( 0.369)	Data  0.000 ( 0.028)	Loss 6.3202e+00 (6.4936e+00)	Acc@1   0.78 (  0.87)	Acc@5   4.69 (  3.66)
Epoch: [1][ 510/1002]	Time  0.361 ( 0.369)	Data  0.000 ( 0.028)	Loss 6.3859e+00 (6.4920e+00)	Acc@1   3.12 (  0.88)	Acc@5   5.47 (  3.67)
Epoch: [1][ 520/1002]	Time  0.361 ( 0.369)	Data  0.000 ( 0.028)	Loss 6.3177e+00 (6.4903e+00)	Acc@1   4.69 (  0.90)	Acc@5   6.25 (  3.71)
Epoch: [1][ 530/1002]	Time  0.366 ( 0.369)	Data  0.000 ( 0.028)	Loss 6.4386e+00 (6.4896e+00)	Acc@1   1.56 (  0.90)	Acc@5   5.47 (  3.71)
Epoch: [1][ 540/1002]	Time  0.364 ( 0.369

Test: [ 80/391]	Time  0.117 ( 0.321)	Loss 6.1722e+00 (6.0764e+00)	Acc@1   0.00 (  2.41)	Acc@5   1.56 (  7.75)
Test: [ 90/391]	Time  0.356 ( 0.318)	Loss 5.4454e+00 (6.0879e+00)	Acc@1   6.25 (  2.25)	Acc@5  28.91 (  7.73)
Test: [100/391]	Time  0.117 ( 0.313)	Loss 6.0092e+00 (6.0768e+00)	Acc@1   0.00 (  2.17)	Acc@5   3.12 (  7.53)
Test: [110/391]	Time  0.827 ( 0.323)	Loss 6.0689e+00 (6.0813e+00)	Acc@1   0.00 (  2.04)	Acc@5   0.00 (  7.12)
Test: [120/391]	Time  0.113 ( 0.324)	Loss 6.3383e+00 (6.0602e+00)	Acc@1   0.78 (  2.19)	Acc@5   0.78 (  7.45)
Test: [130/391]	Time  0.729 ( 0.325)	Loss 5.7307e+00 (6.0688e+00)	Acc@1   3.91 (  2.18)	Acc@5  17.19 (  7.48)
Test: [140/391]	Time  0.114 ( 0.323)	Loss 6.7499e+00 (6.0689e+00)	Acc@1   0.00 (  2.12)	Acc@5   0.00 (  7.35)
Test: [150/391]	Time  1.086 ( 0.328)	Loss 5.5163e+00 (6.0547e+00)	Acc@1   5.47 (  2.22)	Acc@5  14.84 (  7.65)
Test: [160/391]	Time  0.551 ( 0.325)	Loss 6.1299e+00 (6.0375e+00)	Acc@1   0.78 (  2.29)	Acc@5   3.12 (  7.81)
Test: [170

...

Test: [ 30/391]	Time  0.121 ( 0.335)	Loss 2.8292e+00 (2.7528e+00)	Acc@1  46.09 ( 44.48)	Acc@5  71.88 ( 70.21)
Test: [ 40/391]	Time  0.135 ( 0.331)	Loss 3.1085e+00 (2.5622e+00)	Acc@1  36.72 ( 47.64)	Acc@5  63.28 ( 72.43)
Test: [ 50/391]	Time  0.136 ( 0.332)	Loss 1.7177e+00 (2.6538e+00)	Acc@1  57.03 ( 46.35)	Acc@5  85.16 ( 71.09)
Test: [ 60/391]	Time  0.121 ( 0.326)	Loss 2.5559e+00 (2.5542e+00)	Acc@1  35.94 ( 47.55)	Acc@5  75.00 ( 72.43)
Test: [ 70/391]	Time  0.120 ( 0.324)	Loss 2.9608e+00 (2.5748e+00)	Acc@1  35.16 ( 46.18)	Acc@5  65.62 ( 72.15)
Test: [ 80/391]	Time  0.119 ( 0.318)	Loss 2.7883e+00 (2.6085e+00)	Acc@1  35.16 ( 44.61)	Acc@5  64.06 ( 71.43)
Test: [ 90/391]	Time  0.601 ( 0.318)	Loss 2.2815e+00 (2.6115e+00)	Acc@1  42.19 ( 44.18)	Acc@5  80.47 ( 71.37)
Test: [100/391]	Time  0.119 ( 0.310)	Loss 2.5726e+00 (2.5826e+00)	Acc@1  40.62 ( 44.04)	Acc@5  69.53 ( 71.92)
Test: [110/391]	Time  0.116 ( 0.309)	Loss 2.7628e+00 (2.5733e+00)	Acc@1  32.81 ( 43.81)	Acc@5  75.00 ( 72.14)
Test: [120

Epoch: [88][ 300/1002]	Time  0.391 ( 0.396)	Data  0.000 ( 0.030)	Loss 1.6819e+00 (1.7484e+00)	Acc@1  68.75 ( 62.81)	Acc@5  81.25 ( 80.81)
Epoch: [88][ 310/1002]	Time  0.370 ( 0.396)	Data  0.000 ( 0.030)	Loss 1.6585e+00 (1.7488e+00)	Acc@1  59.38 ( 62.77)	Acc@5  85.16 ( 80.80)
Epoch: [88][ 320/1002]	Time  0.395 ( 0.396)	Data  0.000 ( 0.030)	Loss 1.6259e+00 (1.7444e+00)	Acc@1  64.84 ( 62.84)	Acc@5  85.16 ( 80.88)
Epoch: [88][ 330/1002]	Time  0.392 ( 0.396)	Data  0.000 ( 0.030)	Loss 1.6714e+00 (1.7423e+00)	Acc@1  61.72 ( 62.89)	Acc@5  82.03 ( 80.95)
Epoch: [88][ 340/1002]	Time  0.391 ( 0.396)	Data  0.000 ( 0.030)	Loss 1.8714e+00 (1.7430e+00)	Acc@1  60.16 ( 62.89)	Acc@5  77.34 ( 80.90)
Epoch: [88][ 350/1002]	Time  0.387 ( 0.396)	Data  0.000 ( 0.030)	Loss 1.5415e+00 (1.7434e+00)	Acc@1  67.19 ( 62.89)	Acc@5  82.81 ( 80.90)
Epoch: [88][ 360/1002]	Time  0.392 ( 0.396)	Data  0.000 ( 0.030)	Loss 1.5729e+00 (1.7426e+00)	Acc@1  61.72 ( 62.85)	Acc@5  85.16 ( 80.93)
Epoch: [88][ 370/1002]	Time  0.390

Epoch: [88][ 900/1002]	Time  0.394 ( 0.395)	Data  0.000 ( 0.028)	Loss 2.0231e+00 (1.7643e+00)	Acc@1  62.50 ( 62.33)	Acc@5  77.34 ( 80.52)
Epoch: [88][ 910/1002]	Time  0.395 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.6965e+00 (1.7649e+00)	Acc@1  59.38 ( 62.31)	Acc@5  79.69 ( 80.50)
Epoch: [88][ 920/1002]	Time  0.390 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.9603e+00 (1.7646e+00)	Acc@1  57.81 ( 62.32)	Acc@5  78.12 ( 80.50)
Epoch: [88][ 930/1002]	Time  0.389 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.6913e+00 (1.7643e+00)	Acc@1  62.50 ( 62.31)	Acc@5  82.03 ( 80.50)
Epoch: [88][ 940/1002]	Time  0.391 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.6043e+00 (1.7635e+00)	Acc@1  65.62 ( 62.32)	Acc@5  82.81 ( 80.51)
Epoch: [88][ 950/1002]	Time  0.376 ( 0.395)	Data  0.000 ( 0.028)	Loss 2.0622e+00 (1.7638e+00)	Acc@1  54.69 ( 62.32)	Acc@5  76.56 ( 80.51)
Epoch: [88][ 960/1002]	Time  0.400 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.6539e+00 (1.7643e+00)	Acc@1  60.16 ( 62.31)	Acc@5  81.25 ( 80.51)
Epoch: [88][ 970/1002]	Time  0.397

Epoch: [89][ 170/1002]	Time  0.395 ( 0.397)	Data  0.000 ( 0.034)	Loss 1.6165e+00 (1.7331e+00)	Acc@1  64.06 ( 62.82)	Acc@5  83.59 ( 80.88)
Epoch: [89][ 180/1002]	Time  0.385 ( 0.397)	Data  0.000 ( 0.034)	Loss 1.5894e+00 (1.7316e+00)	Acc@1  67.19 ( 62.86)	Acc@5  84.38 ( 80.87)
Epoch: [89][ 190/1002]	Time  0.389 ( 0.397)	Data  0.000 ( 0.033)	Loss 2.0375e+00 (1.7343e+00)	Acc@1  57.81 ( 62.89)	Acc@5  77.34 ( 80.82)
Epoch: [89][ 200/1002]	Time  0.390 ( 0.397)	Data  0.000 ( 0.033)	Loss 1.9298e+00 (1.7312e+00)	Acc@1  60.94 ( 63.03)	Acc@5  76.56 ( 80.87)
Epoch: [89][ 210/1002]	Time  0.393 ( 0.397)	Data  0.000 ( 0.033)	Loss 1.9885e+00 (1.7382e+00)	Acc@1  57.03 ( 62.86)	Acc@5  78.12 ( 80.74)
Epoch: [89][ 220/1002]	Time  0.394 ( 0.396)	Data  0.000 ( 0.032)	Loss 1.7217e+00 (1.7368e+00)	Acc@1  61.72 ( 62.89)	Acc@5  82.81 ( 80.79)
Epoch: [89][ 230/1002]	Time  0.387 ( 0.396)	Data  0.000 ( 0.032)	Loss 1.9334e+00 (1.7385e+00)	Acc@1  66.41 ( 62.89)	Acc@5  76.56 ( 80.77)
Epoch: [89][ 240/1002]	Time  0.389

Epoch: [89][ 770/1002]	Time  0.386 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.7493e+00 (1.7560e+00)	Acc@1  60.16 ( 62.46)	Acc@5  86.72 ( 80.53)
Epoch: [89][ 780/1002]	Time  0.389 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.8365e+00 (1.7567e+00)	Acc@1  62.50 ( 62.47)	Acc@5  76.56 ( 80.52)
Epoch: [89][ 790/1002]	Time  0.392 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.6838e+00 (1.7563e+00)	Acc@1  64.84 ( 62.47)	Acc@5  80.47 ( 80.53)
Epoch: [89][ 800/1002]	Time  0.413 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.8347e+00 (1.7571e+00)	Acc@1  60.94 ( 62.46)	Acc@5  76.56 ( 80.51)
Epoch: [89][ 810/1002]	Time  0.393 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.8822e+00 (1.7579e+00)	Acc@1  60.16 ( 62.44)	Acc@5  76.56 ( 80.49)
Epoch: [89][ 820/1002]	Time  0.391 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.6596e+00 (1.7585e+00)	Acc@1  60.16 ( 62.45)	Acc@5  81.25 ( 80.49)
Epoch: [89][ 830/1002]	Time  0.388 ( 0.395)	Data  0.000 ( 0.028)	Loss 1.8577e+00 (1.7582e+00)	Acc@1  64.06 ( 62.45)	Acc@5  75.78 ( 80.49)
Epoch: [89][ 840/1002]	Time  0.392

In [5]:
import torchvision.models as torch_models
import torch
from collections import OrderedDict

model = torch_models.resnet18()
    
checkpoint = torch.load('/home/jtang/Desktop/original_model/checkpoint.pth.tar')